<a href="https://colab.research.google.com/github/erlichsefi/ScrapeAnything/blob/main/JavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# // Get all elements in the HTML page\
# const elements = document.getElementsByTagName('*');

# // Create an array to store the element details
# const elementDetails = [];

# // Iterate through each element
# for (let i = 0; i < elements.length; i++) {
#   const element = elements[i];

#   // Get the bounding rectangle of the element
#   const rect = element.getBoundingClientRect();

#   // Get the text content of the element
#   const textContent = element.textContent.trim();

#   // Get the tooltip value if it exists
#   const tooltip = element.hasAttribute('title') ? element.getAttribute('title') : '';

#   // Store the element, its bounding rectangle, text content, and tooltip details
#   const elementInfo = {
#     element: element,
#     rect: rect,
#     textContent: textContent,
#     ariaLabel: element.hasAttribute('aria-label') ? element.getAttribute('aria-label') : '',
#     tooltip: tooltip,
#     e_type: element.nodeName
#   };
#   if (elementInfo.rect.width > 0 && elementInfo.rect.height > 0){
#   // Add the element details to the array
#    if (elementInfo.tooltip != '' || elementInfo.textContent != '' || elementInfo.ariaLabel != '')
#     elementDetails.push(elementInfo);
#   }
# }

# let parents = elementDetails.map(e=>e.element.parentElement)
# let withoutParents = elementDetails.filter(elementDetail=> !parents.includes(elementDetail.element))

# // Display the element details
# console.log("X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel")
# console.log(withoutParents.map( e=> e.rect.x+","+e.rect.y+","+e.rect.width+","+e.rect.height+","+e.e_type+","+e.rect.textContent+","+e.tooltip+","+e.ariaLabel).join("\n"));

In [ ]:

on_screen_data = """
X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel
VM1301:41 464.40625,18,33.953125,24,A,undefined,,Gmail (opens a new tab)
513.359375,18,42.640625,24,A,undefined,,Search for Images (opens a new tab)
575,10,40,40,A,undefined,,Google apps
623,10,40,40,A,undefined,,Google Account: Sefi Erlich  
(erlichsefi@gmail.com)
202.5,258,272,92,DIV,undefined,,
61.5,377,33,46,DIV,undefined,,
94.5,388,443,27,TEXTAREA,undefined,Search,Search
541.5,377,40,44,DIV,undefined,,Search by voice
581.5,377,40,44,DIV,undefined,,Search by image
197.59375,453,127.40625,36,INPUT,undefined,,Google Search
336.890625,453,142.5,36,INPUT,undefined,,I'm Feeling Lucky
353.953125,516,38.109375,15,A,undefined,,
395.6875,516,34.09375,15,A,undefined,,
0,875,677,48,DIV,undefined,,
24.328125,923,66.59375,46,A,undefined,,
90.921875,923,99.265625,46,A,undefined,,
190.1875,923,86.8125,46,A,undefined,,
277,923,146.703125,46,A,undefined,,
428.03125,923,75.90625,46,A,undefined,,
503.9375,923,68.125,46,A,undefined,,
572.0625,923,80.59375,46,DIV,undefined,,
"""